In [4]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

train_ratio = 0.85
n_epochs = 100

horizon_pred = 1

# params = {
#     'seq_length' : [5, 15, 30],
#     'batch_size' : [20, 30, 50],
#     'units_layer1' : [50, 100, 200, 300],
#     'activation_layer1' : ['relu', 'sigmoid'],
#     'kernel_reg_factor' : [None, 0.01, 0.001, 0.0001], 
#     'dropout_layer1' : [None, 0.1, 0.2],
#     'dense_mid_size' : [None, 10, 20, 30],
#     'activation_output' : ['relu'],
#     'optim' : ['adam', 'SGD', 'RMSprop'],
#     'company_inputs' : [['PETR4.SA'], ['PETR4.SA', 'BZ=F', '6L=F']],
# }

params = {
    'seq_length' : [20],
    'batch_size' : [10],
    'units1' : [120, 125, 130],
    'activation1' : ['relu'], 
    'kernel_reg_factor1' : [0.001, 0.0001],
    'dropout_layer1' : [0.05, 0.10, 0.15],
    'dense_mid_size' : [80],
    'activation_out' : ['relu'],
    'optim' : ['nadam'],
    'company_inputs' : [['PETR4.SA']],
}

company_output = 'PETR4.SA'
end_date = '2024-10-30' 
start_date = '2020-01-01'


In [5]:
from sklearn.model_selection import ParameterGrid
from parametrized_training_model import parametrized_training

import mlflow
import mlflow.tensorflow

import numpy as np
from sklearn.metrics import root_mean_squared_error, r2_score

mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
mlflow.set_experiment("LSTM Fix1 Grid Search Test 12")

param_grid = ParameterGrid(params)
mlflow.tensorflow.autolog()

for param_sample in param_grid:
    with mlflow.start_run():

        mlflow.log_params(param_sample)
        mlflow.log_param("n_epochs", n_epochs)
        mlflow.log_param("train_ratio", train_ratio)
        mlflow.log_param("horizon_pred", horizon_pred)
        
        (scaler, generator_train, generator_test, model) = parametrized_training(param_sample['company_inputs'], company_output, start_date, end_date, train_ratio,
                            horizon_pred, param_sample['seq_length'], param_sample['batch_size'],
                            param_sample['units1'], param_sample['activation1'], l2(param_sample['kernel_reg_factor1']), 
                            param_sample['dropout_layer1'], param_sample['dense_mid_size'], param_sample['activation_out'], param_sample['optim'])

        hist = model.fit(generator_train, epochs=n_epochs, validation_data=generator_test)

        # def make_predictions(generator, scaled_X):
        y_pred = []
        y_valid = []
        for k in range(len(generator_test)):
            print(k)
            X_scaled_temp, y_scaled_temp = generator_test[k]
            X_scaled_temp = X_scaled_temp.reshape(len(X_scaled_temp),param_sample['seq_length'])
            y_pred_scaled = model.predict(X_scaled_temp)
            y_pred.append(scaler.inverse_transform(y_pred_scaled.reshape(-1,1)))
            
            y_valid.append(scaler.inverse_transform(y_scaled_temp.reshape(-1,1)))

        y_pred = np.concatenate(y_pred, axis=0 )
        y_valid = np.concatenate(y_valid, axis=0 )

        mlflow.tensorflow.log_model(model, "model")

        # Calculate RMSE and R² score
        rmse = root_mean_squared_error(y_valid, y_pred)
        r2 = r2_score(y_valid, y_pred)

        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2score", r2)

        mlflow.end_run()


2024/11/23 22:05:05 INFO mlflow.tracking.fluent: Experiment with name 'LSTM Fix1 Grid Search Test 12' does not exist. Creating a new experiment.
2024/11/23 22:05:05 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
[*********************100%***********************]  1 of 1 completed
2024/11/23 22:05:11 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.sequence.TimeseriesGenerator'>. Dataset logging skipped.
2024/11/23 22:05:11 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.sequence.TimeseriesGenerator'>. Dataset logging skipped.
c:\GitRepos\TC4-Thales\tech_challenge_4\training_model\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_ada

Epoch 1/100
 99/101 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0485

101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.0476 - val_loss: 0.0071
Epoch 2/100
 98/101 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0020

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0019 - val_loss: 0.0037
Epoch 3/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0015

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0015 - val_loss: 0.0018
Epoch 4/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0015 - val_loss: 0.0039
Epoch 5/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0013 - val_loss: 0.0024
Epoch 6/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0017 - val_loss: 0.0027
Epoch 7/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 8/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0014 - val_loss: 0.0033
Epoch 9/100
 97/101 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0011

...

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 6.8148e-04 - val_loss: 4.2105e-04
Epoch 61/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 5.4549e-04 - val_loss: 7.3324e-04
Epoch 62/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.7287e-04 - val_loss: 6.5880e-04
Epoch 63/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.8197e-04

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 5.8312e-04 - val_loss: 4.1426e-04
Epoch 64/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6.8733e-04 - val_loss: 5.1326e-04
Epoch 65/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.7113e-04 - val_loss: 6.2980e-04
Epoch 66/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.6358e-04 - val_loss: 4.3178e-04
Epoch 67/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6.9240e-04 - val_loss: 0.0033
Epoch 68/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 8.2644e-04 - val_loss: 5.0004e-04
Epoch 69/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.6853e-04 - val_loss: 5.0524e-04
Epoch 70/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6.4450e-04 - val_loss: 5.2191e-04
Epoch 71/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.7511e-04 - val_loss: 9.5502e-04
Epoch 72/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 8.5885e-04 - val_loss: 6.7501e-04
Epoch 73/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 10

2024/11/23 22:48:10 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: unsupported operand type(s) for *: 'int' and 'slice'
2024/11/23 22:48:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/23 22:48:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step


2024/11/23 22:48:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/23 22:48:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/23 22:48:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run honorable-cub-158 at: http://127.0.0.1:7777/#/experiments/840909497958366180/runs/1d1daf28e17749f59e1367980779c298.
2024/11/23 22:48:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:7777/#/experiments/840909497958366180.


In [6]:
# from sklearn.metrics import root_mean_squared_error, r2_score

# rmse = root_mean_squared_error(y_pred, y_valid_f)
# r2 = r2_score(y_pred, y_valid_f)

# print(f'RMSE: {rmse:.4f}')
# print(f'R² Score: {r2:.4f}')

In [7]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(range(len(y_gen_train)),y_gen_train, label="Training Data")
# plt.plot(range(len(y_gen_train),len(y_gen_train)+len(y_gen_test)),y_gen_test, label="Test Data")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [8]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(y_pred, label="Predicted Prices")
# plt.plot(y_valid_f, label="Real Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [9]:
# import mlflow
# import mlflow.tensorflow

# mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
# mlflow.set_experiment("LSTM Experiments")

# from sklearn.metrics import root_mean_squared_error, r2_score

# mlflow.tensorflow.autolog()
# with mlflow.start_run():
#     mlflow.tensorflow.log_model(model, "model")

#     # Calculate RMSE and R² score
#     rmse = root_mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)

#     print(f'RMSE: {rmse:.4f}')
#     print(f'R² Score: {r2:.4f}')

#     mlflow.log_metric("RMSE", rmse)
#     mlflow.log_metric("R2-Score", r2)

In [10]:
# # Predict using the trained model
# future_steps = 10  # Number of days to predict
# future_data = data[-n_steps:].reshape(-1, n_steps, 1)
# predicted_prices = []

# for i in range(future_steps):
#     prediction = model.predict(future_data)[0, 0]
#     predicted_prices.append(prediction)
#     future_data = np.roll(future_data, -1, axis=1)
#     future_data[0, -1] = prediction

# # Inverse transform the predicted prices to original scale
# predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1)).flatten()

In [11]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(predicted_prices, label="Predicted Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()